# Simulating Molecules using VQE- The Li-H

In this notebook, we will be designing our own implementation of a variational quantum eigensolver (VQE) algorithm that simulates the ground state energy of the Lithium Hydride (LiH) molecule. 
Also, as already mentioned in the challenge notebook, throughout this challenge, we will be able to make choices on how we want to compose our simulation from available options.

## Contents
1. VQE Implementation in Qiskit
   1. Introduction
   2. [Running VQE on a Noisy Simulator](#implementationnoisy)

2. [References](#references)

## A. Introduction<a id='introduction'></a>
In many applications it is important to find the minimum eigenvalue of a matrix. For example, in chemistry, the minimum eigenvalue of a Hermitian matrix characterizing the molecule is the ground state energy of that system. In the future, the quantum phase estimation algorithm may be used to find the minimum eigenvalue. However, its implementation on useful problems requires circuit depths exceeding the limits of hardware available in the NISQ era. Thus, in 2014, Peruzzo *et al.* proposed VQE to estimate the ground state energy of a molecule using much shallower circuits [1]. 

Formally stated, given a Hermitian matrix $H$ with an unknown minimum eigenvalue $\lambda_{min}$, associated with the eigenstate $|\psi_{min}\rangle$, VQE provides an estimate $\lambda_{\theta}$ bounding $\lambda_{min}$:

\begin{align*}
    \lambda_{min} \le \lambda_{\theta} \equiv \langle \psi(\theta) |H|\psi(\theta) \rangle
\end{align*}  

where $|\psi(\theta)\rangle$ is the eigenstate associated with $\lambda_{\theta}$. By applying a parameterized circuit, represented by $U(\theta)$, to some arbitrary starting state $|\psi\rangle$, the algorithm obtains an estimate $U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$ on $|\psi_{min}\rangle$. The estimate is iteratively optimized by a classical controller changing the parameter $\theta$ minimizing the expectation value of $\langle \psi(\theta) |H|\psi(\theta) \rangle$.







### B. Running VQE on a Noisy Simulator<a id='implementationnoisy'></a>

Here, we calculate the ground state energy for LiH using a noisy simulator.
By changing the parameter inter_dist, we can use our VQE algorithm to calculate the ground state energy of LiH at various interatomic distances.
Among the choices on the challenge notebook, one combination would out perform others and give us the lowest estimation of LiH ground state energy with the quickest convergence.


In [ ]:
# Importing necessary account settings
%matplotlib inline
import numpy as np
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import matplotlib.pyplot as plt
# Loading the IBM Q account(s)
provider = IBMQ.load_account()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# All the necessary libraries
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

from qiskit.providers.aer import noise

In [ ]:
# Defining our molecule
inter_dist=[]
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0' + str(inter_dist), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')


In [ ]:
# Reduce the problem size
freeze_list = [0]
remove_list = [-3, -2] 

In [ ]:
# Get the noisy simulator!
backend = Aer.get_backend('qasm_simulator')
chip_name = 'ibmq_essex'
device = provider.get_backend(chip_name)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device.properties())
basis_gates = noise_model.basis_gates
quantum_instance =  QuantumInstance(backend=backend, shots=10000, 
                                   noise_model=noise_model, 
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30)

In [ ]:
# Define the classical solver
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref

# Define your function for computing the qubit operations of LiH
def compute_LiH_qubitOp(inter_dist, basis='sto3g'):
    
    # Specify details of our molecule
    driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 ' + str(inter_dist), unit=UnitsType.ANGSTROM, charge=0,
                         spin=0, basis=basis)

    # Compute relevant 1 and 2 body integrals.
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))

    # Please be aware that the idx here with respective to original idx
    freeze_list = [0]
    remove_list = [-3, -2] # negative number denotes the reverse order
    
    # Prepare full idx of freeze_list and remove_list
    # Convert all negative idx to positive
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    
    # Update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]

    # Prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
    # and if PARITY mapping is selected, there will be reduction of qubits
    energy_shift = 0.0
    map_type = 'parity'
    qubit_reduction = True if map_type == 'parity' else False

    ferOp = FermionicOperator(h1=h1, h2=h2)
    if len(freeze_list) > 0:
        ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
        num_spin_orbitals -= len(freeze_list)
        num_particles -= len(freeze_list)
    if len(remove_list) > 0:
        ferOp = ferOp.fermion_mode_elimination(remove_list)
        num_spin_orbitals -= len(remove_list)

    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
    qubitOp.chop(10**-10)
    
    shift = energy_shift + nuclear_repulsion_energy

    return qubitOp, num_spin_orbitals, num_particles, qubit_reduction, shift

# Run the VQE on noisy system now
distances = np.arange(0.4, 4.0, 0.4)
exact_energies = []
vqe_energies = []
optimizer = SPSA(max_trials=500)

for inter_dist in distances:
    qubitOp, num_spin_orbitals, num_particles, qubit_reduction, shift = compute_LiH_qubitOp(inter_dist)
    ref = exact_solver(qubitOp)
    result = NumPyEigensolver(qubitOp).run()   
    exact_energies.append(np.real(result.eigenvalues) + shift)
    init_state = HartreeFock(num_spin_orbitals, num_particles, qubit_mapping='parity')
    var_form = RYRZ(num_qubits=qubitOp.num_qubits,depth=2)
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(inter_dist, 2), "VQE Result:", vqe_result, "Exact Energy:",exact_energies[-1])
print("All energies have been calculated")

As the effects of noise increase as the number of two qubit gates circuit depth increase, we use a heuristic variational form (RYRZ) rather than UCCSD as RYRZ has a much shallower circuit than UCCSD and uses substantially fewer two qubit gates.




In [ ]:
# For plotting part-1: Ground state energy at various inter-atomic distances
plt.plot(distances, exact_energies, label="Exact Energy")
plt.scatter(distances, vqe_energies, label="VQE Energy", color='red')
plt.plot(distances, vqe_energies, color='black')
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.title('Exact molecular energy and VQE simulation (The simulation curve, with noise)')
plt.legend()
plt.show()

In [ ]:
# Energy convergence of specific optimization algorithm used- SPSA
# Fix a particular inter-atomic distance and check the energy minimisation as a function of the no.of iterations of Optimization performed
qubitOp, num_spin_orbitals, num_particles, qubit_reduction, shift= compute_LiH_qubitOp(inter_dist=1.6)
# Here, check for 250 iterations
for i in range(0,251,10):
    optimizer= SPSA(max_trials=i)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues)+shift )
    initial_state = init_state
    var_form = var_form
    # Solve for VQE
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(quantum_instance)['eigenvalue']+shift )
    vqe_energies.append(vqe_result)
    print("SPSA iteration value:",i, "Interatomic Distance:", np.around(inter_dist, decimals=2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All iterations have been performed")

In [ ]:
# For plotting part-2: Energy Errors of VQE procedure for RYRZ
import numpy as np
eval_counts= np.arange(0,250,10)
Energy_Error= [m - n for m,n in zip(exact_energies, vqe_energies)] 
plt.plot(eval_counts, Energy_Error, label="SPSA")
plt.xlabel('eval_counts')
plt.ylabel('Energy Errors for SPSA optimizer')
plt.title('Energy Errors for VQE procedure for RYRZ')
plt.legend()
plt.show()

# Hartree limit=  chemical accuracy (defined as being within 0.0016 Hartree of the exact result).

## References<a id='references'></a>
1. Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." *Nature communications* 5 (2014): 4213.
2. Griffiths, David J., and Darrell F. Schroeter. Introduction to quantum mechanics. *Cambridge University Press*, 2018.
3. Shende, Vivek V., Igor L. Markov, and Stephen S. Bullock. "Minimal universal two-qubit cnot-based circuits." arXiv preprint quant-ph/0308033 (2003).
4. Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." Nature 549.7671 (2017): 242.

In [ ]:
import qiskit
qiskit.__qiskit_version__